# Heston-stochastic-volatility-model

$
\DeclareMathOperator*{\E}{\mathbb{E}}
\DeclareMathOperator*{\Q}{\mathbb{Q}}
$

In the notebooks I'll discuss various implementations of the Heston stochastic volatility model of pricing European options. This stochastic volatility model is described by two stochastic differential equations: one for the stock process, and another one for the volatility process both driven by their respective Brownian motions:


\begin{align} 
dS_t & = \sqrt{v_t} S_t dB_t, \qquad S_0 > 0 \\
dv_t & = \kappa (\theta-r_t) dt + \sigma\sqrt{v_t}(\rho dB_t + \sqrt{1-\rho^2} dW_t), \qquad v_0 > 0.
\end{align}

The $S_t$ stock price process is driven $B_t$ Brownian motion, and the volatility, $v_t$, is driven by $W_t$ Brownian motion mixed with $B_t$ so that the volatility is driven by a Brownian motion that has $\rho$ correlation with $B_t$. $W_t$ is independent of $B_t$.

The stock price process is similar to a geometric Brownian motion except that its volatility is a stochastic process instead of being constant or deterministic.

The volatility process is a CIR process, a mean reversion process which has mean-reversion level $\theta$ and the volatility mean revert to this level at the mean-reversion speed $\kappa$. $\sigma$ is volatility of the volatility process (vol of vol).

$B$ and $W$ are two independent $\Q$-Brownian motions generating the filtration $F_t$

The time is in the interval $[0, T]$ where $T$ is the expiry time of the option measured in years.

